In [1]:
!pip install PyPDF2

     |████████████████████████████████| 81kB 2.3MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=27d6623e063a61d86b72e22bfd7ca3f45b62b0bab6600e5b8b8c5403db93f09f
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [2]:
import PyPDF2 as pdf
import requests 
url = 'https://canvas.harvard.edu/courses/63146/files/8208924/download?verifier=yKeGz92l9DvL6BodDw9MzzlqkIYBBV9FLeN8niwu&wrap=1'
r = requests.get(url, allow_redirects=True)

filename = 'testFileDownloadedFromInternet.pdf'
open(filename, 'wb').write(r.content)

143447

In [3]:
file = open(filename, 'rb')
pdf_reader = pdf.PdfFileReader(file)
pdf_reader

In [4]:
help(pdf_reader)

Help on PdfFileReader in module PyPDF2.pdf object:

class PdfFileReader(builtins.object)
 |  Initializes a PdfFileReader object.  This operation can take some time, as
 |  the PDF stream's cross-reference tables are read into memory.
 |  
 |  :param stream: A File object or an object that supports the standard read
 |      and seek methods similar to a File object. Could also be a
 |      string representing a path to a PDF file.
 |  :param bool strict: Determines whether user should be warned of all
 |      problems and also causes some correctable problems to be fatal.
 |      Defaults to ``True``.
 |  :param warndest: Destination for logging warnings (defaults to
 |      ``sys.stderr``).
 |  :param bool overwriteWarnings: Determines whether to override Python's
 |      ``warnings.py`` module with a custom implementation (defaults to
 |      ``True``).
 |  
 |  Methods defined here:
 |  
 |  __init__(self, stream, strict=True, warndest=None, overwriteWarnings=True)
 |      Initialize

In [5]:
pdf_reader.getNumPages()

4

In [6]:
page1 = pdf_reader.getPage(0)
page1.extractText()

'Draft Syllabus for \nCSCI \nE-7 - Introduction to Programing in Python\n Instructor: Jeff Parker\n On Line Course \nÐ Live lecture will be recorded for distance students\n Meets \nTuesdays\n, 5:50 in Maxwell\n-Dworkin G115\n.  First day of class: September 3, 2019\n Course\n Description\n Python is \na language with a simple syntax\n and a powerful set of libraries. It is an interpreted \nlanguage, with a rich programming environment, including a robust debugger and profiler. While \nit is easy for beginners to learn, it is widely use\nd in many scientific areas for data exploration. \nThis course is an introduction to the Python programming language for students without prior \n\nprogramming experience. \n We cover data types, control flow, and introduce the analysis of program performance. Th\ne examples and problems used in this course are drawn from diverse areas such as text processing \nand simple graphics creation. Students enrolled for graduate credit implement a final project

# Write out just the first page to a new file

In [7]:
pdf_writer = pdf.PdfFileWriter()
pdf_writer.addPage(page1)

output = open('1Page.pdf','wb')
pdf_writer.write(output)
output.close()

# Read back in the 1 page pdf

In [8]:
pdf_reader = pdf.PdfFileReader('1Page.pdf')
page1 = pdf_reader.getPage(0)

In [9]:
!pip install nltk

In [10]:
# Array Operations
import numpy as np

# For dataframe
import pandas as pd

# For English Stopwords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# For Tf-idf Weighting
from sklearn.feature_extraction.text import TfidfVectorizer

# For Regular Expressions
import re

def preprocess_text(text):
    tokens = word_tokenize(text)
    words = []
    for word in tokens:
        # Replace non-alpha character with space
        word = re.sub('[^A-Za-z]+', ' ', word)
        words.append(word.lower())
    return " ".join(words)

In [11]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
extracted_text = page1.extractText()
preprocess_text(extracted_text)

# NLTK Stop words 
stop_words = stopwords.words('english')

# Some extra Stop words like text in Header and Footer of every page
stop_words.extend([ "com", "etc", "www"])

vectorizer = TfidfVectorizer(#max_features = 500, 
                             stop_words = stop_words, 
                             preprocessor=preprocess_text)
X = vectorizer.fit_transform([extracted_text])
words = vectorizer.get_feature_names()
weights = X.toarray()

In [17]:
ordered_weights = weights[0][(np.argsort(-weights[0]))]
weight_ordered_words = np.array(words)[np.argsort(-weights[0])]

In [18]:
weight_ordered_words[:10]

array(['students', 'python', 'programming', 'language', 'line', 'course',
       'use', 'command', 'text', 'class'], dtype='<U21')

In [19]:
d = {"words": weight_ordered_words, "weight": ordered_weights}
df = pd.DataFrame(data = d, columns = ["words", "weight"])

df

,words,weight
0,students,0.355830
1,python,0.254164
2,programming,0.254164
3,language,0.254164
4,line,0.203331
...,...,...
140,editor,0.050833
141,dworkin,0.050833
142,due,0.050833
143,draft,0.050833
